# **UB** : Save and Restore Crystal Orientation

see: https://github.com/bluesky/hklpy/issues/50

**Objectives**

1. Save the information defining the crystal orientation into the descriptor
1. Restore crystal orientation from a given Bluesky run
1. List runs that have orientation that can be restored

In [1]:
import gi
gi.require_version('Hkl', '5.0')

from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import bluesky.preprocessors as bpp
import databroker
import hkl
from hkl.calc import A_KEV
from hkl.util import Lattice, run_orientation_info, list_orientation_runs
from hkl.geometries import *
import numpy as np
import pyRestTable
from ophyd import Component, Device, EpicsSignal, Signal
from ophyd.signal import AttributeSignal, ArrayAttributeSignal
from ophyd.sim import *
import pandas as pd

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2

RE = RunEngine({})
RE.subscribe(bec)
RE.subscribe(cat.v1.insert)
RE.md["notebook"] = "UB_save_restore"
RE.md["objective"] = "Demonstrate UB matrix save & restore"

/home/mintadmin/Apps/anaconda/envs/bluesky_2021_1/lib/python3.8/site-packages/databroker/core.py:1856: UserWarning: There are 2 entrypoints for the databroker handler spec 'NPY_SEQ'. They are [EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')), EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', None)]. The match EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')) has won the race.
  warnings.warn(


-------------

-------------

## Build a simulated 4-circle diffractometer

In [4]:
class Fourc(SimulatedE4CV):
    pass

fourc = Fourc("", name="fourc")
fourc.energy.put(A_KEV / 1.54)
a0 = 5.4310196
fourc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
fourc.calc.sample.compute_UB(
    fourc.calc.sample.add_reflection(4, 0, 0, (-145.451, 0, 0, 69.0966)),
    fourc.calc.sample.add_reflection(0, 4, 0, (-145.451, 0, 90, 69.0966))
)
fourc.pa()

orange = Fourc("", name="orange")
orange.pa()

===================== ===========================================================================
term                  value                                                                      
===================== ===========================================================================
diffractometer        fourc                                                                      
geometry              E4CV                                                                       
class                 Fourc                                                                      
energy (keV)          8.05092                                                                    
wavelength (angstrom) 1.54000                                                                    
calc engine           hkl                                                                        
mode                  bissector                                                                  
positions           

In [5]:
class Kappa(SimulatedK4CV):
    pass

kappa = Kappa("", name="kappa")
kappa.energy.put(A_KEV / 1.54)
a0 = 5.4310196
kappa.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
kappa.calc.sample.compute_UB(
    kappa.calc.sample.add_reflection(4, 0, 0, (55.4507, 0, 90, -69.0966)), 
    kappa.calc.sample.add_reflection(0, 4, 0, (-1.5950, 134.7568, 123.3554, -69.0966))
)
kappa.pa()

===================== =================================================================================
term                  value                                                                            
===================== =================================================================================
diffractometer        kappa                                                                            
geometry              K4CV                                                                             
class                 Kappa                                                                            
energy (keV)          8.05092                                                                          
wavelength (angstrom) 1.54000                                                                          
calc engine           hkl                                                                              
mode                  bissector                                 

In [6]:
class Sixc(SimulatedE6C):
    pass

sixc = Sixc("", name="sixc")
sixc.energy.put(A_KEV / 1.54)
a0 = 5.4310196
sixc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
sixc.calc.sample.compute_UB(
    sixc.calc.sample.add_reflection(4, 0, 0, (0, -145.451, 0, 0, 0, 69.0966)),
    sixc.calc.sample.add_reflection(0, 4, 0, (0, -145.451, 90, 0, 0, 69.0966))
)
sixc.pa()

===================== ========================================================================================================
term                  value                                                                                                   
===================== ========================================================================================================
diffractometer        sixc                                                                                                    
geometry              E6C                                                                                                     
class                 Sixc                                                                                                    
energy (keV)          8.05092                                                                                                 
wavelength (angstrom) 1.54000                                                                                  

In [7]:
def scan_all():
    yield from bp.count([noisy_det])
    yield from bp.count([noisy_det, fourc])
    yield from bp.count([noisy_det, fourc, orange, kappa, sixc])
    yield from bp.scan([noisy_det], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, fourc], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, kappa], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], sixc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, sixc], sixc.h, 0.9, 1.1, 2)

In [8]:
_uids = RE(scan_all())



Transient Scan ID: 1     Time: 2021-04-25 12:45:02
Persistent Unique Scan ID: '60522e70-c99c-4c6a-8325-a92315671bbb'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 12:45:02.3 |      1.090 |
+-----------+------------+------------+
generator count ['60522e70'] (scan num: 1)





Transient Scan ID: 2     Time: 2021-04-25 12:45:02
Persistent Unique Scan ID: 'b50a9ad1-57aa-4610-b908-e9cb872611b5'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k |    fourc_l |  noisy_det |
+-----------+------------+------------+------------+------------+------------+
|         1 | 12:45:02.5 |      0.000 |      0.000 |      0.000 |      0.985 |
+-----------+------------+------------+------------+------------+------------+
generator count ['b50a9ad1'] (scan num: 2)





Transient Scan ID: 3     Time

In [9]:
run = cat[5]
run.primary.config["fourc"].read()

<xarray.Dataset>
Dimensions:                    (dim_0: 6, dim_1: 6, dim_10: 6, dim_11: 21, dim_2: 3, dim_3: 3, dim_4: 3, dim_5: 3, dim_6: 2, dim_7: 3, dim_8: 4, dim_9: 4, time: 2)
Coordinates:
  * time                       (time) float64 1.619e+09 1.619e+09
Dimensions without coordinates: dim_0, dim_1, dim_10, dim_11, dim_2, dim_3, dim_4, dim_5, dim_6, dim_7, dim_8, dim_9
Data variables: (12/21)
    fourc_energy               (time) float64 8.051 8.051
    fourc_energy_units         (time) <U3 'keV' 'keV'
    fourc_energy_offset        (time) int64 0 0
    fourc_geometry_name        (time) <U4 'E4CV' 'E4CV'
    fourc_class_name           (time) <U5 'Fourc' 'Fourc'
    fourc_sample_name          (time) <U7 'silicon' 'silicon'
    ...                         ...
    fourc__hklpy_version       (time) <U25 '0.3.15+169.gdf976c7.dirty' '0.3.1...
    fourc__pseudos             (time, dim_7) <U1 'h' 'k' 'l' 'h' 'k' 'l'
    fourc__reals               (time, dim_8) <U5 'omega' 'chi' ... 'phi' 'tth'
    fourc__constraints         (time, dim_9, dim_10) <U23 'omega' ... 'False'
    fourc__mode                (time) <U9 'bissector' 'bissector'
    fourc_orientation_attrs    (time, dim_11) <U19 'orientation_attrs' ... '_...

In [10]:
fourc.show_constraints()

===== ========= ========== ======================= ==== ========
axis  low_limit high_limit value                   fit  inverted
===== ========= ========== ======================= ==== ========
omega -180.0    180.0      -8.97224432640064       True False   
chi   -180.0    180.0      1.3179200548037037e-109 True False   
phi   -180.0    180.0      0.0007000000002791517   True False   
tth   -180.0    180.0      -17.94448865280128      True False   
===== ========= ========== ======================= ==== ========



In [11]:
import pprint
pprint.pprint(run_orientation_info(cat[2]))

{'fourc': {'U': [[-1.2217304763832569e-05, -0.9999999999253688, 0.0],
                 [0.0, 0.0, 1.0],
                 [-0.9999999999253688, 1.2217304763832569e-05, 0.0]],
           'UB': [[-1.4134287010388982e-05,
                   -1.156907131528427,
                   7.084099625231898e-17],
                  [0.0, 0.0, 1.1569071316147683],
                  [-1.156907131528427,
                   1.4134287010459822e-05,
                   7.083926530138442e-17]],
           '_constraints': [['omega',
                             '-180.0',
                             '180.0',
                             '0.0',
                             'True',
                             'False'],
                            ['chi', '-180.0', '180.0', '0.0', 'True', 'False'],
                            ['phi', '-180.0', '180.0', '0.0', 'True', 'False'],
                            ['tth', '-180.0', '180.0', '0.0', 'True', 'False']],
           '_hklpy_version': '0.3.15+169.gdf976c7.dirty'

In [12]:
list_orientation_runs(cat, "energy", "energy_units", "lattice")

,scan_id,sample_name,diffractometer_name,geometry_name,energy,energy_units,lattice,uid
0,2,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",b50a9ad
1,3,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",c38d3bc
2,3,silicon,kappa,K4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",c38d3bc
3,3,silicon,sixc,E6C,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",c38d3bc
4,3,main,orange,E4CV,8.000000,keV,"[1.54, 1.54, 1.54, 90.0, 90.0, 90.0]",c38d3bc
5,5,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",dee760e
6,7,silicon,kappa,K4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",0bb738e
7,9,silicon,sixc,E6C,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",a891c80


------------

TODO: show series of scans with different orientation reflections and different **UB**